## 语料处理

In [9]:
# 根据词性标注语料生成用于实体识别的训练语料
def tag_line(words):
    """
    根据输入的分词序列（元素为：词 + 词性），标记主体.
    words: ['[', '香港/ns', '特别/a', '行政区', ']']
    遇到中括号 + ns 标注的，则将中括号内的全部词汇整体提取，否则只提取ns部分
    """
    chars = []  # 记录单个字
    tags = []  # 记录实体状态 B M E S O
    temp_word = ""  # 拼接实体词组（括号内的多个词）
    for word in words:
        word = word.strip()
        w, h = word.split("/")
        if temp_word == "":
            bracket_pos = word.find("[")
            # 词中不含 [，则对词进行标注
            if bracket_pos == -1:
                l = len(w)
                if l == 0:
                    continue
                chars.extend(w)  # 自动将 w转为列表
                # 如果词性为 ns， 则标注
                if h == "ns":
                    tags += ["S"] if l == 1 else ["B"] + ["M"] * (l - 2) + ["E"]
                else:
                    tags += ["O"] * l
            # 词中含有 [
            else:
                w = w[bracket_pos + 1: ]  # 取中括号之后的部分，而且一般括号都在头部，不会出现漏词
                temp_word += w
        
        # 括号内实体已存在， 则寻找右括号
        else:
            bracket_pos = word.find("]")
            # 未找到右括号，则继续拼接
            if bracket_pos == -1:
                temp_word += w
            # 找到右括号，停止拼接，且标注括号内部分
            else:
                w = w[: bracket_pos]  # 取括号左边的剩余词
                w = temp_word + w  # 最后将拼接词转给 w
                h = word[bracket_pos + 1: ]  # 取中括号后面的词性
                temp_word = ""  # 清空临时词组
                l = len(w)
                if l == 0:
                    continue
                chars.extend(w)
                if h == "ns":  # 说明临时词组是地名
                    tags += ["S"] if l == 1 else ["B"] + ["M"] * (l - 2) + ["E"]
                else:  # 说明临时词组不是地名
                    tags += ["O"] * l
                        
    assert temp_word == "", "临时拼接词组不为空"
    return chars, tags              

In [18]:
## Test!
text = "在/p １９９８年/t 来临/v 之际/f ，/w 我/r 十分/m 高兴/a 地/u 通过/p [中央/n 人民/n 广播/vn 电台/n]nt 、/w [中国/ns 国际/n 广播/vn 电台/n]nt 和/c [中央/n 电视台/n]nt ，/w 向/p 全国/n 各族/r 人民/n ，/w 向/p [香港/ns 特别/a 行政区/n]ns 同胞/n 、/w 澳门/ns 和/c 台湾/ns 同胞/n 、/w 海外/s 侨胞/n ，/w 向/p 世界/n 各国/r 的/u 朋友/n 们/k ，/w 致以/v 诚挚/a 的/u 问候/vn 和/c 良好/a 的/u 祝愿/vn ！/w "
words = text.strip().split()
chars, tags = tag_line(words)
import pandas as pd
df = pd.DataFrame(list(zip(chars, tags)), columns=["char", "tag"])
df[df["tag"] != "O"]

,char,tag
52,香,B
53,港,M
54,特,M
55,别,M
56,行,M
57,政,M
58,区,E
62,澳,B
63,门,E
65,台,B


In [97]:
# 语料处理函数
def corpusHandler(corpusPath):
    """
    利用tag_line函数对语料进行批量处理
    params:
        corpusPath: 语料地址
    """
    import os
    root = os.path.dirname(corpusPath)  # 返回文件所在目录
    train_path = os.path.join(root, "train.txt")  # 设置训练文件（待写入）
    test_path = os.path.join(root, "test.txt")  # 设置测试文件（待写入）
    
    with open(corpusPath, mode="r", encoding="UTF-8") as corpus_f, \
        open(train_path, mode="w", encoding="UTf-8") as train_f, \
        open(test_path, mode="w", encoding="UTF-8") as test_f:
        pos = 0
        for line in corpus_f:
            line = line.strip("\r\t\n ")
            if line == "":
                continue
            saveObj = test_f if pos % 5 == 0 else train_f  # 用来划分训练集和测试集  4:1
            words = line.split()[1:]  # 第一个词是新闻时间，舍弃
            if len(words) == 0:
                continue
            line_chars, line_tags = tag_line(words)
            for char, tag in zip(line_chars, line_tags):
                line = char + "\t" + tag + "\n"
                saveObj.write(line)
            saveObj.write("\n")  # 一行语料处理结束， 输出多写一个 "\n"
            pos += 1

In [100]:
corpusPath = "F:/for learn/Python/NLP_in_Action/chapter-4/data/people-daily.txt"
corpusHandler(corpusPath)

## 预测结果检测

In [117]:
import pandas as pd
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report

In [111]:
path = "C:/Users/Cigar/Documents/jupyter/NLP_learn/NLP_In_Action/Chap04_save/"
result = pd.read_table(path + "test.rst", header=None, names=["char", "y_true", "y_pred"])
result[:5]

,char,y_true,y_pred
0,迈,O,O
1,向,O,O
2,充,O,O
3,满,O,O
4,希,O,O


In [116]:
recall_states = ["B", "M", "E", "S"]
y_true = result["y_true"].apply(lambda x: x in recall_states).astype(int)
y_pred = result["y_pred"].apply(lambda x: x in recall_states).astype(int)

In [118]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    347270
           1       0.92      0.85      0.89     11650

   micro avg       0.99      0.99      0.99    358920
   macro avg       0.96      0.93      0.94    358920
weighted avg       0.99      0.99      0.99    358920

